<a href="https://colab.research.google.com/github/ELehmann91/Thesis_Multilingual_Transferlearning/blob/master/Text_Annotation_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Colaboratory logo" height="45px" src="/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

<h1>What is Colaboratory?</h1>

Colaboratory, or "Colab" for short, allows you to write and execute Python in your browser, with 
- Zero configuration required
- Free access to GPUs
- Easy sharing

Whether you're a **student**, a **data scientist** or an **AI researcher**, Colab can make your work easier. Watch [Introduction to Colab](https://www.youtube.com/watch?v=inN8seMm7UI) to learn more, or just get started below!

# ECOICOP Annotation
  
Use this tool to annotate your product texts with the correspondant eCoiCop category. 


## Get code from GitHub
  
At first we need to download the code from Github to here.

In [0]:
%%capture
!pip install eli5
!git clone 'https://github.com/ELehmann91/Thesis_Multilingual_Transferlearning'

%cd Thesis_Multilingual_Transferlearning
import labeler_cc5
import coicop_model
import pandas as pd
import numpy as np
from tqdm import tqdm
import io

## Upload CSV / Excel from Local Drive

To upload from your local drive, start with the following code:

In [81]:
from google.colab import files
uploaded = files.upload()

Saving eantoclassifyECB.csv to eantoclassifyECB (1).csv


It will prompt you to select a file. Click on “Choose Files” then select and upload the file. Wait for the file to be 100% uploaded. You should see the name of the file once Colab has uploaded it.
Finally, type in the following code to import it into a dataframe (make sure the filename matches the name of the uploaded file).

In [0]:
#stata
df = pd.read_stata('table_BCE_finale.dta')

In [0]:
df = pd.read_csv(io.BytesIO(uploaded['eantoclassifyECB.csv']),sep=',')
#df = pd.read_excel(io.BytesIO(uploaded['carrfour_trans_pred.xlsx']),encoding='ANSI')
#df = pd.read_excel(uploaded['carrfour_trans_pred.xlsx'],encoding='unicode')

In [83]:
# Dataset is now stored in a Pandas Dataframe
#df['productCategory'] = df['productCategory'].apply(lambda x: str(x).replace('suesses','süßes').replace('ue','ü').replace('ae','ä').replace('oe','ö'))
print(df.shape)
df[:2]

(132639, 3)


,ean,title_accent,id
0,30080751,"coolwave bonbons sans sucres avec édulcorants,...",1811805
1,30080768,"cool wave fraise, sans sucres, la boite de 28g",1811806


## Upload from Google Drive

Loading CSV files stored in your google drive.

In [2]:
from google.colab import drive, files

drive.mount('/content/gdrive')
path ='/content/gdrive/My Drive/Thesis_ecb_ecoicop'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


The commands will bring you to a Google Authentication step. You should see a screen with Google Drive File Stream wants to access your Google Account. After you allow permission, copy the given verification code and paste it in the box in Colab.
In the notebook, click on the charcoal > on the top left of the notebook and click on Files. Locate the data folder you created earlier and find your data. Right-click on your data and select Copy Path. Store this copied path into a variable and you are ready to go.

Specify the exact path to your file and the seperator of your file to read it in.

In [3]:
import pandas as pd

data_path = '/data/'#
#file_path = 'fra/carrfour_trans_pred.csv'
file_path = 'normalized/norm_fr.csv'
df = pd.read_csv(path+data_path+file_path,sep='|',index_col=False)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,3,4,5,6,7,8,9,10,11,12,13,14,16,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.shop.value_counts()

ecb                 133639
Carrefour            90454
Auchan               25511
banque_de_france     14910
carrefour             8610
17.75                    2
Name: shop, dtype: int64

In [0]:
df = df[df['shop']=='banque_de_france']

Print out the dataframe to see if the import is correct.

In [6]:
df[:1]

,Unnamed: 0,lang,name,categ,prod_desc,text_other,url,words_from_url,unit,cc3,cc4,cc5,cc3_pred,cc4_pred,cc5_pred,shop,brand,price,id,labeld_by
193227,186523,fr,Fromage de brebis crème à tartiner SOCIETE,cremerie,NaN,NaN,https://www.carrefour.fr/p/fromage-de-brebis-c...,fromage brebis creme tartiner societe,NaN,NaN,NaN,1145_Cheese and curd,11_Food,"114_Milk, cheese and eggs",1145_Cheese and curd,banque_de_france,NaN,NaN,30005501,NaN


# Predict CoiCop Level 4
  
How does it work? - Word Embeddings (left)
* Words are translated in vectors which are learned represent the words
* Embeddings are a multidimensional space (often 300+ dimesnsion) and words which relate to each other are closer in this space
* Those spaces can be aligned for different languages
* The example shows a three dimensional space with embeddings for three different languages, the translation are idealy close neighbours and the distance between milk and cheese is closer than the distance betwenn milk and potato  
more information: https://towardsdatascience.com/introduction-to-word-embeddings-4cf857b12edc
  
How does it work? - Recurrent Neural Network (right)
* Now every word is represented by a vector and the input to the model is a sequence of vectors
* LSTM (Long Short Term Memory) Networks are good in solving sequential tasks, because they are able to remember information from previous states (words) and output a representation for the sentence 
* This representation is used to classify in to the coicop categories  
more information: https://towardsdatascience.com/a-beginners-guide-on-sentiment-analysis-with-rnn-9e100627c02e 




![embed](https://drive.google.com/uc?id=1AoleK5q47ZTkPCtvxDD6icEpcnLc9viE)

For the prediction you have to specify the names of certain columns in your dataframe which the tool will use.  
* name_col is the column with the product name (mandatory)
* category_col is the column with the category given by the supermarked (optional)
* url_col is the column with  the url to the product (optional)
* lang ist the language of the texts, supports 'de' and 'fr' (mandatory)
* label_cat5 is the column with labels,if labeld (optional)



In [11]:
# init predictor
  
name_col = 'name'
category_col = 'categ'
url_col = 'url'
lang = 'fr'
label_cat5 = 'cc5' #'coicop4_str'

 #coicop_model #
CoiCop_Predictor = coicop_model.predictor(df
                                          , name_col
                                          , category_col
                                          , url_col
                                          , label_cat5
                                          , lang)


using name, category and words in url as input
using french embeddings
95% quantile no. of words per row is 18 (trained on 39)


In [12]:
# execute to create predictions and store them in the object
CoiCop_Predictor.predict()

100%|██████████| 149/149 [00:11<00:00, 13.07it/s]

predictions ready


if products are already labeled you can test the consistency with the prediction.  
label_cat5 is specified two lines above

In [13]:
if label_cat5 is not None:
    CoiCop_Predictor.test_performance()
    CoiCop_Predictor.confusion_matrix()

number of observation (labeled / all): 14429 / 14910 consistency  98.0 %


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                                precision    recall  f1-score   support

                                                     1111_Rice       1.00      1.00      1.00       244
                                                    1113_Bread       0.99      0.97      0.98       285
                                    1114_Other bakery products       0.88      0.98      0.93        65
                                         1115_Pizza and quiche       1.00      1.00      1.00        16
                              1116_Pasta products and couscous       1.00      1.00      1.00       770
                                        1117_Breakfast cereals       0.98      0.99      0.99       395
                                            1121_Beef and veal       0.98      0.96      0.97       179
                                                     1122_Pork       0.87      1.00      0.93        20
                                                  1124_Poultry 

If you need not only the predicted label, but the probabilities for every class:

In [0]:
if False:
    probs = CoiCop_Predictor.predict_proba()

## Explonation  
  
It shows the three best candidates for a product and highlights the words which contribute to the estimate (positiv: green, negative: red)
If you leave the the brackets empty it will show a random product. If you want the explonation for a particular product, you can enter the text.  
If you enter text it always needs to be in marks ''.

In [15]:
text = None
categ= '1211_Coffee'
#text = ''

CoiCop_Predictor.tell_me_why(text,categ)

prediction 1211_Coffee
label 1211_Coffee


Explanation(estimator="SGDClassifier(alpha=0.001, average=False, class_weight=None,\n              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,\n              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,\n              n_iter_no_change=5, n_jobs=None, penalty='elasticnet',\n              power_t=0.5, random_state=RandomState(MT19937) at 0x7FEF3DF56780,\n              shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,\n              warm_start=False)", description=None, error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='1211_Coffee', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='[0] Café', weight=7.59814837202199, std=None, value=1.0), FeatureWeight(feature='[7] bio', weight=0.15227010953343062, std=None, value=1.0), FeatureWeight(feature='[6] grain', weight=0.1217498341173934, std=None, value=1.0), FeatureWeight(feature='[1] en', weight=0.10625680153284472, std=None, value=1.0), FeatureWeight(feature='[8] naturela', weight=0.10247408545160125, std=None, value=1.0), FeatureWeight(feature='[2] grain', weight=0.09546987056613773, std=None, value=1.0), FeatureWeight(feature='[3] bio', weight=0.09197251371984028, std=None, value=1.0), FeatureWeight(feature='[4] NATURELAepicerie', weight=0.08117351735640266, std=None, value=1.0), FeatureWeight(feature='[5] sucreecafe', weight=0.03133860932317689, std=None, value=1.0)], neg=[FeatureWeight(feature='<BIAS>', weight=-3.9344130728877076, std=None, value=1.0)], pos_remaining=0, neg_remaining=0), proba=0.9038406066871352, score=4.44644064073511, weighted_spans=WeightedSpans(docs_weighted_spans=[DocWeightedSpans(document='Café en grain bio  NATURELAepicerie-sucreecafe grain bio naturela', spans=[('Café', [(0, 4)], 7.59814837202199), ('en', [(5, 7)], 0.10625680153284472), ('grain', [(8, 13)], 0.09546987056613773), ('bio', [(14, 17)], 0.09197251371984028), ('NATURELAepicerie', [(19, 35)], 0.08117351735640266), ('sucreecafe', [(36, 46)], 0.03133860932317689), ('grain', [(47, 52)], 0.1217498341173934), ('bio', [(53, 56)], 0.15227010953343062), ('naturela', [(57, 65)], 0.10247408545160125)], preserve_density=False, vec_name=None)], other=FeatureWeights(pos=[FeatureWeight(feature=<FormattedFeatureName 'Highlighted in text (sum)'>, weight=8.380853713622818, std=None, value=None)], neg=[FeatureWeight(feature='<BIAS>', weight=-3.9344130728877076, std=None, value=1.0)], pos_remaining=0, neg_remaining=0)), heatmap=None), TargetExplanation(target='1152_Margarine and other vegetable fats', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='[7] bio', weight=0.9064999575647864, std=None, value=1.0), FeatureWeight(feature='[6] grain', weight=0.6230970635226597, std=None, value=1.0), FeatureWeight(feature='[2] grain', weight=0.596184411117542, std=None, value=1.0), FeatureWeight(feature='[8] naturela', weight=0.58911734750003, std=None, value=1.0), FeatureWeight(feature='[3] bio', weight=0.43421160123919544, std=None, value=1.0), FeatureWeight(feature='[1] en', weight=0.3509193721662078, std=None, value=1.0), FeatureWeight(feature='[4] NATURELAepicerie', weight=0.2865671904804051, std=None, value=1.0), FeatureWeight(feature='[5] sucreecafe', weight=0.2519345125618529, std=None, value=1.0)], neg=[FeatureWeight(feature='<BIAS>', weight=-5.834115587886433, std=None, value=1.0), FeatureWeight(feature='[0] Café', weight=-2.472589424349803, std=None, value=1.0)], pos_remaining=0, neg_remaining=0), proba=0.012631849732587686, score=-4.268173556083556, weighted_spans=WeightedSpans(docs_weighted_spans=[DocWeightedSpans(document='Café en grain bio  NATURELAepicerie-sucreecafe grain bio naturela', spans=[('Café', [(0, 4)], -2.472589424349803), ('en', [(5, 7)], 0.3509193721662078), ('grain', [(8, 13)], 0.596184411117542), ('bio', [(14, 17)], 0.43421160123919544), ('NATURELAepicerie', [(19, 35)], 0.2865671904804051), ('sucreecafe', [(36, 46)], 0.2519345125618529), ('grain', [(47, 52)], 0.6230970635226597), ('

Get the data bach from the predictor and find the new columns ['cc3_pred','cc4_pred','cc5_pred'] which store the predicted labels.

In [16]:
df = CoiCop_Predictor.get_df()
df[:1]

,Unnamed: 0,lang,name,categ,prod_desc,text_other,url,words_from_url,unit,cc3,cc4,cc5,cc3_pred,cc4_pred,cc5_pred,shop,brand,price,id,labeld_by,url_text,text
193227,186523,fr,Fromage de brebis crème à tartiner SOCIETE,cremerie,NaN,NaN,https://www.carrefour.fr/p/fromage-de-brebis-c...,fromage brebis creme tartiner societe,NaN,NaN,NaN,1145_Cheese and curd,11_Food,"114_Milk, cheese and eggs",1145_Cheese and curd,banque_de_france,NaN,NaN,30005501,NaN,fromage brebis creme tartiner societe,Fromage de brebis crème à tartiner SOCIETEcrem...


# Annotation

For the annotation tool you have to specify the naes of certain columns in your dataframe which the tool will use.  
* Labeled_by takes your name and stores it after every item you labeld.  
* Text column 1 is the name of the column for the first line of text which will be displayed to help you labeling the product, usually this is the product name.  
* Text column 2 is the name of the column for the second line of text which will be displayed to help you labeling the product, this could be the category or the translation.  
* The URL column is the name of the column where the url of the scraped product is located (if it is in the dataframe). The url might be helpfull if you are unsure about the product nd want to look up the whole page on the website. If you do not have the url in your data you can leave it empty.  
* CoiCop 5 prediction is the column where the prediction is stored. The prediction will appear preselected in the dropdown menu.   
  
The data are sorted in a way that catogies with few labels come first, if predictions of those label are available.


In [0]:
# init labeler

labeled_by = 'Erik'
text_column_1 = 'name'
text_column_2 = 'categ'
url_column = 'url'
CoiCop_5_pred_col = 'cc5_pred'

CoiCop_Labeler = labeler_cc5.labeler( labeled_by
                        , df
                        , text_column_1
                        , text_column_2
                        , url_column
                        , CoiCop_5_pred_col)

Now you are ready to label, after executing the next line you will see:
* Text 1
* Text 2
* URL
* Dropdown with one category which is the prediction for the product. If you are dissatisfied with it, you can click on the dropdown and select the right one.
* Next-Button will skip this product
* Save-Button will annotate the selected category to your dataframe
  
If you finished labeling just jump to the next line.

In [18]:
CoiCop_Labeler.start_to_label()

The next line will print out your current progress of annotating the data.

In [19]:
CoiCop_Labeler.get_stats()

new labels: 2
in total 14431 of 14910 labeled ( 97.0 %)


Output the data including annotation.

In [20]:
df_new = CoiCop_Labeler.output_labels()
df_new[:1]

,lang,name,categ,prod_desc,text_other,url,words_from_url,unit,cc3,cc4,cc5,cc3_pred,cc4_pred,cc5_pred,shop,brand,price,id,labeld_by,url_text,text,labeled_by,sort_columns
0,fr,Huile soleil cheveux ULTRA DOUX,hygiene-et-beaute,NaN,NaN,https://www.carrefour.fr/p/huile-soleil-cheveu...,huile soleil cheveux ultra doux,NaN,NaN,NaN,NaN,99_Non-Food,999_Non-Food,9999_Non-Food,banque_de_france,NaN,NaN,3600541676596,NaN,huile soleil cheveux ultra doux,Huile soleil cheveux ULTRA DOUXhygiene-et-beau...,None,49


## Save data to Local Drive

In [0]:
from google.colab import files

df.to_csv('df_out.csv',sep='|')
files.download('df_out.csv')

## Save data to Google Drive

In [87]:

if False:
    print(path+data_path+file_path,'saved')
    df.to_csv(path+data_path+file_path,sep='|')

/content/gdrive/My Drive/Thesis_ecb_ecoicop/data/fra/eantoclassifyECB.csv saved
